In [1]:
import scipy.stats
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle("accidents.pkl.gz")
df_filtered = df.loc[df["p36"].isin([0,1]),["p13a", "p36"]]

In [3]:
df_hyp1 = pd.DataFrame({
    'road_type': df_filtered['p36'],
    'fatal': df_filtered['p13a']>0
}).dropna()
ct = pd.crosstab(df_hyp1["road_type"], df_hyp1["fatal"])
print(ct)

fatal      False  True
road_type             
0          24293   166
1          78618   911


In [4]:
chi,p,degrees,expected = scipy.stats.chi2_contingency(ct)
print('Chi:', chi)
print('P:', p)
print('Expected:', expected)

Chi: 39.314921530329016
P: 3.6067450279444316e-10
Expected: [[24205.67901104   253.32098896]
 [78705.32098896   823.67901104]]


Hodnota korelačního koeficientu je menší než 0.05, korelace mezi typem silnice (pouze 1. třídy a dálnice) a fatálními nehodami tedy existuje. Nehody na silnicích 1. třídy byly fatální častěji, než je očekáváno (dle hodnot expected a reálných hodnot)

In [32]:
df_skoda = df[["p45a", "p53"]].dropna()
skoda = df.loc[df["p45a"]==39]
audi = df.loc[df["p45a"]==2]
result = scipy.stats.ttest_ind(skoda["p53"],audi["p53"])
print(result)




Ttest_indResult(statistic=-23.391701716453692, pvalue=9.24412736332601e-121)
